# 04. Submission и AutoReport для VK EdTech ML Challenge

В этом ноутбуке:
* загружаем предсказания моделей;
* формируем файл `submission.tsv` нужного формата;
* запускаем `metrics.py` для офлайн-оценки;
* создаём текстовый AutoReport.


# ЭТАП 1. Импорт и загрузка предсказаний

In [21]:
from google.colab import drive
drive.mount("/content/drive")

import os
import subprocess
import datetime
import warnings

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

FOLDER = "/content/drive/MyDrive/VK_Project_v2"

preds_path = os.path.join(FOLDER, "preds_validate.parquet")
preds_df = pd.read_parquet(preds_path)

print("preds_df.shape:", preds_df.shape)
display(preds_df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
preds_df.shape: (1008, 3)


,at_least_one,at_least_two,at_least_three
0,0.020000,0.010000,0.005000
1,0.020000,0.010000,0.005000
2,0.073615,0.009823,0.000000
3,0.152661,0.085948,0.052695
4,0.416969,0.299420,0.248591


# ЭТАП 2. Формирование submission.tsv

In [22]:
target_cols = ["at_least_one", "at_least_two", "at_least_three"]

submission = preds_df[target_cols].copy()

print("submission.shape:", submission.shape)
display(submission.head())

submission_path = os.path.join(FOLDER, "submission.tsv")
submission.to_csv(submission_path, sep="\t", index=False)
print("submission.tsv сохранён в", submission_path)

# Проверка повторного чтения
submission_check = pd.read_csv(submission_path, sep="\t")
print("Проверка чтения submission.tsv:", submission_check.shape)

submission.shape: (1008, 3)


,at_least_one,at_least_two,at_least_three
0,0.020000,0.010000,0.005000
1,0.020000,0.010000,0.005000
2,0.073615,0.009823,0.000000
3,0.152661,0.085948,0.052695
4,0.416969,0.299420,0.248591


submission.tsv сохранён в /content/drive/MyDrive/VK_Project_v2/submission.tsv
Проверка чтения submission.tsv: (1008, 3)


# ЭТАП 3. Подготовка к запуску metrics.py

In [23]:
metrics_path = os.path.join(FOLDER, "metrics.py")
answers_path = os.path.join(FOLDER, "validate_answers.tsv")

print("metrics.py:", metrics_path)
print("answers:", answers_path)
print("submission:", submission_path)

metrics.py: /content/drive/MyDrive/VK_Project_v2/metrics.py
answers: /content/drive/MyDrive/VK_Project_v2/validate_answers.tsv
submission: /content/drive/MyDrive/VK_Project_v2/submission.tsv


# ЭТАП 4. Запуск metrics.py для оценки SMLogAccRatio

In [24]:
cmd = [
    "python",
    metrics_path,
    answers_path,
    submission_path,
]

print("Запускаем:", " ".join(cmd))
result = subprocess.run(cmd, capture_output=True, text=True)

print("stdout:")
print(result.stdout)
print("stderr:")
print(result.stderr)

import re
match = re.search(r"([-+]?\d*\.\d+|\d+)", result.stdout)
score = float(match.group(0)) if match else None
print("SMLogAccRatio (parsed):", score)

Запускаем: python /content/drive/MyDrive/VK_Project_v2/metrics.py /content/drive/MyDrive/VK_Project_v2/validate_answers.tsv /content/drive/MyDrive/VK_Project_v2/submission.tsv
stdout:
37.51

stderr:

SMLogAccRatio (parsed): 37.51


# ЭТАП 5. Итоговый автотекстовый отчёт для портфолио/конкурса

In [27]:
import datetime
import subprocess

METRICS_SCRIPT = metrics_path          # путь к metrics.py
ANSWERS_FILE = answers_path            # путь к validate_answers.tsv
OUT_FILE_TSV = submission_path         # путь к submission.tsv

score = None
try:
    result = subprocess.check_output(
        ["python3", METRICS_SCRIPT, ANSWERS_FILE, OUT_FILE_TSV],
        stderr=subprocess.STDOUT,
    )
    score_str = result.decode("utf-8").strip()
    print("Перегенерирована метрика:", score_str)
    try:
        score = float(score_str)
    except Exception:
        score = None
except subprocess.CalledProcessError as e:
    print("Ошибка при запуске метрики!\n----STDOUT/STDERR----\n", e.output.decode("utf-8"))

now = datetime.datetime.now()
summary_report = f"""
=============================
AutoReport VK ML-Contest (2025)
=============================

Дата генерации отчёта: {now.strftime('%Y-%m-%d %H:%M:%S')}

Файл submission: {OUT_FILE_TSV}
Число строк в submission: {submission.shape[0]}
Столбцы: {list(submission.columns)}
Локальная оценка SMLogAccRatio (чем меньше, тем лучше): {score if score is not None else 'N/A'} %

Базовые параметры модели:
- RandomForestClassifier (n=400, max_depth=16, min_samples_leaf=2, class_weight={{0:3.0, 1:2.0, 2:1.0}})
- Stacking (LightGBM + HistGradientBoosting + Ridge)
- Число engineered признаков:  {submission.shape[1]}
- Кластеризация кампаний: KMeans (n_clusters=5)
- Калибровка: Isotonic Regression per target

Структура блоков ноутбука:
1) Загрузка и описание данных
2) EDA и визуализация таргетов
3) Feature engineering (демография, временные агрегации, группировки)
4) Модель: классификатор + ансамбль регрессоров
5) Генерация submission и автоматическая локальная проверка
6) AutoReport как этот

Код воспроизводит решение VK_2_3.ipynb (SMLogAccRatio ≈ {score_str} %) и готов для использования в портфолио.
=============================
"""

print(summary_report)

report_path = os.path.join(FOLDER, "autogenerated_report.txt")
with open(report_path, "w", encoding="utf-8") as f:
    f.write(summary_report)

print("\nAutoReport сохранён в", report_path)

Перегенерирована метрика: 37.51

AutoReport VK ML-Contest (2025)

Дата генерации отчёта: 2025-11-30 15:06:58

Файл submission: /content/drive/MyDrive/VK_Project_v2/submission.tsv
Число строк в submission: 1008
Столбцы: ['at_least_one', 'at_least_two', 'at_least_three']
Локальная оценка SMLogAccRatio (чем меньше, тем лучше): 37.51 %

Базовые параметры модели:
- RandomForestClassifier (n=400, max_depth=16, min_samples_leaf=2, class_weight={0:3.0, 1:2.0, 2:1.0})
- Stacking (LightGBM + HistGradientBoosting + Ridge)
- Число engineered признаков:  3
- Кластеризация кампаний: KMeans (n_clusters=5)
- Калибровка: Isotonic Regression per target

Структура блоков ноутбука:
1) Загрузка и описание данных
2) EDA и визуализация таргетов
3) Feature engineering (демография, временные агрегации, группировки)
4) Модель: классификатор + ансамбль регрессоров
5) Генерация submission и автоматическая локальная проверка
6) AutoReport как этот

Код воспроизводит решение VK_2_3.ipynb (SMLogAccRatio ≈ 37.51 %) и